# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] <urlopen error [Errno 101] Network is unreachable>


False

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import re, pickle

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
df = pd.read_sql_table('Disaster_DB', con=engine)
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[A-Za-z]|[0-9]|[$-._@+&]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    url = re.findall(url_regex, text)
    for u in url:
        text = text.replace(u, 'urlplaceholder')
        
    token = word_tokenize(text)
    lemmatize = WordNetLemmatizer()
    
    clean_token = []
    for tok in token:
        clean_token.append(lemmatize.lemmatize(tok).lower().strip())
    
    return clean_token

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
pipeline.fit(X_train.values, y_train.values)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = pipeline.predict(X_test.values)

In [9]:
def display_results(y_test, y_pred):  
    id = 0
    results = pd.DataFrame(columns=['Category', 'Precision', 'Recall', 'F1-Score'])
    for col in y_test:
        precision, recall, f1, support = precision_recall_fscore_support(y_test[col], y_pred[:,id],average='weighted')
        results.at[id+1, 'Category'] = col
        results.at[id+1, 'Precision'] = precision
        results.at[id+1, 'Recall'] = recall
        results.at[id+1, 'F1-Score'] = f1
        id = id+1
    print('Average Precision: ', results['Precision'].mean())
    print('Average Recall: ', results['Recall'].mean())
    print('Average f1: ', results['F1-Score'].mean())
    
    return results

In [9]:
results = display_results(y_test, y_pred)
results

Average Precision:  0.9406589755915219
Average Recall:  0.9499455076507258
Average f1:  0.9363722522966437


/home/srnvsn/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Category,Precision,Recall,F1-Score
1,request,0.89597,0.89823,0.884434
2,offer,0.990866,0.995423,0.993139
3,aid_related,0.775928,0.774947,0.769371
4,medical_help,0.913955,0.927067,0.896255
5,medical_products,0.952004,0.955752,0.937204
6,search_and_rescue,0.968547,0.973604,0.961849
7,security,0.962219,0.980928,0.971483
8,military,0.957968,0.967348,0.953374
9,child_alone,1,1,1
10,water,0.946232,0.949191,0.934235


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [10]:
def build_new_model():   
    parameters = {
        #'vect__ngram_range': ((1, 1),(1,2)),
        #'vect__max_df' : (0.5, 0.75, 1.0),
        #'tfidf__use_idf': (True, False),
        #'vect__max_features': (None, 5000, 10000),
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [11]:
model_new = build_new_model()
model_new.fit(X_train.values, y_train.values)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [13]:
model_new.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
y_pred = model_new.predict(X_test.values)
results = display_results(y_test, y_pred)
results

Average Precision:  0.9413636078135036
Average Recall:  0.950115523780461
Average f1:  0.9363409807510783


/home/srnvsn/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Category,Precision,Recall,F1-Score
1,request,0.898777,0.900366,0.887001
2,offer,0.990866,0.995423,0.993139
3,aid_related,0.781793,0.780134,0.774472
4,medical_help,0.9126,0.92661,0.894952
5,medical_products,0.951144,0.955447,0.936522
6,search_and_rescue,0.974591,0.973909,0.962023
7,security,0.962219,0.980928,0.971483
8,military,0.96378,0.967348,0.952302
9,child_alone,1,1,1
10,water,0.943897,0.947818,0.931882


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
pipeline_mod = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])
pipeline_mod.fit(X_train.values, y_train.values)
y_pred = pipeline_mod.predict(X_test.values)
results = display_results(y_test, y_pred)
results

Average Precision:  0.9349356479070365
Average Recall:  0.9365621866689918
Average f1:  0.9356964595503021


,Category,Precision,Recall,F1-Score
1,request,0.85496,0.852914,0.8539
2,offer,0.99114,0.990845,0.990992
3,aid_related,0.693755,0.69469,0.69417
4,medical_help,0.90235,0.90235,0.90235
5,medical_products,0.937562,0.938206,0.937881
6,search_and_rescue,0.953765,0.960177,0.956681
7,security,0.968636,0.971773,0.970183
8,military,0.959274,0.962771,0.960828
9,child_alone,1,1,1
10,water,0.956668,0.956668,0.956668


### 9. Export your model as a pickle file

In [16]:
pickle.dump(model_new, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.